## 고급 HTML 분석
---
- 웹 스크레이핑은 복잡한 웹 페이지에서 필요한 정보를 얻어낸다는 점에서는 미켈란젤로와 비슷한 방식을 따라야 한다.
- 원하지 않는 콘텐츠를 깍아내어 필요한 정보를 얻는 방법은 여러가지가 있다.

- 이 장에서는 복잡한 HTML 페이지를 분석해서 원하는 정보만 추출하는 방법에 대해 알아본다.

### 2.1 닭 잡는 데 소 잡는 칼을 쓸 필요는 없다.
---
- 복잡한 태그를 만나면 당장 여러 줄의 코드를 써서라도 필요한 정보만을 추출하고 싶다는 생각이 들 것이다.
- 하지만 이 장에서 소개하는 테크닉을 부주의하게 사용하면 코드는 디버그하기 어려워지거나, 취약해지거나, 둘 다가 될 수 있다.
- 시작하기 전, 고급 HTML 분석을 쓰지 않아도 필요한 결과를 얻을 수 있는 방법을 몇 가지 알아본다.

- 다음과 비슷한 코드를 짰다고 하자.

In [1]:
bs.findAll('table')[4].find_all('tr')[2].find_all('div')[1].find('a')

NameError: name 'bs' is not defined

- 별로 좋아보이지 않고, 간결함이나 우아함은 찾아볼 수 없을 뿐더러, 사이트 관리자가 사이트를 조금만 수정하더라도 웹 스크레이퍼의 동작이 멈출 수 있다.
- 위 코드는 신뢰할 수 없고, 사이트 구조가 절대 변하지 않는다는 가정하에 만들어져 있다.

- 그럼 어떻게 해야할까?
  - '페이지 인쇄'같은 링크를 찾아보거나, 더 나은 HTML 구조를 갖춘 모바일 버전 사이트를 찾아보라.
  - 자바스크립트 파일에서 숨겨진 정보를 찾아라.
    - 물론 이렇게 하려면 자바스크립트 파일을 불러와 분석해야 한다.
    - 필자는 사이트에 포함된 구글 맵스를 조작하는 자바스크립트를 살펴본 후 위도와 경도가 포함된 거리 주소를 깔끔하게 배열로 정리된 형태로 수집한 적이 있다.
  - 중요한 정보는 페이지 타이틀에 있을 때가 대부분이지만, 원하는 정보가 페이지 URL에 들어 있을 때도 있다.
  - 원하는 정보가 오직 이 웹사이트에만 있다면 할 수 있는 일이 더는 없을 수 있다.
    - 그렇지 않다면, 이 정보를 다른 소스에서 가져올 수는 없는지 생각해보라.
    - 다른 웹사이트에 같은 데이터가 있지는 않을까? 이 웹사이트에 있는 데이터가 혹시 다른 웹사이트에서 수집한 것은 아닐까?

- 데이터가 깊숙이 파문혀 있거나 정형화되지 않을수록, 코드부터 짜려고 달려들었다간 빠져나올 수 없는 구멍에 파묻히게 된다.
- 대안이 없다고 확신한다면, 이 장의 내용이 도움이 될 것이다.

- 이 장에서는 태그의 위치, 문맥, 속성, 콘텐츠에 따라 태그를 선택하는 표준적이고 창의적인 방법을 설명한다.

- 이 장에서 설명하는 내용을 숙지하고 정확히 사용한다면 더 안정적이고 신뢰할 수 있는 웹 크롤러를 만들 수 있게 된다.

### 2.2 다시 BeautifulSoup
---
- 태그 분석하기

In [1]:
<span class="green"></span>
<span class="red"></span>

<span class="red">Heavens! what a virulent attack!</span> replied <span class="green">
    the prince
</span>, not in the least disconcerted by this reception.

SyntaxError: invalid syntax (3051113286.py, line 1)

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html, 'html.parser')

nameList = bs.findAll('span', {'class': 'green'})
for name in nameList:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


### 2.2.1 find()와 findAll()
---
- find()와 findAll()은 BeautifulSoup에서 가장 자주 쓰는 함수이다.
- 이 함수를 쓰면 HTML 페이지에서 원하는 태그를 다양한 속성에 따라 쉽게 필터링할 수 있다.

In [2]:
# 두 함수는 거의 비슷한데, BeautifulSoup 문서의 함수 정의만 봐도 알 수 있다.
# findAll(tag, attributes, recursive, text, limit, keywords)
# find(tag, attributes, recursive, text, keywords)

In [3]:
# 실제로 이 함수를 쓸 때는 거의 항상 처음 두 매개변수인 tag, attributes만 쓰게 될 것이다.

- tag 매개변수는 태그 이름인 문자열을 넘기거나, 태그 이름으로 이루어진 파이썬 리스트를 넘길 수도 있다.

In [ ]:
# 문서의 헤더 태그 리스트를 반환한다.
bs.findAll({'h1', 'h2', 'h3', 'h4', 'h5', 'h6'})

- attributes 매개변수는 속성으로 이루어진 파이썬 딕셔너리를 받고, 그중 하나에 일치하는 태그를 찾는다.

In [4]:
# HTML 문서에서 녹색과 적색 span 태그를 모두 반환한다.
bs.findAll('span', {'class': {'green', 'red'}})

NameError: name 'bs' is not defined

- recursive 매개변수는 문서에서 얼마나 깊이 찾아 들어가고 싶은지 지정하는 불리언이다.
- recursion이 True이면 findAll 함수는 매개변수에 일치하는 태그를 찾아 자식, 자식의 자식을 검색한다.
- False이면 문서의 최상위 태그만 찾는다.
- 기본적으로 findAll은 재귀적으로(recursive=True) 동작한다.
- 일반적으로 이 옵션은 그대로 두는 것이 좋다.

- text 매개변수는 태그의 속성이 아니라 텍스트 콘텐츠에 일치한다는 점이 다르다.
- 예를 들어 예제 페이지에서 태그에 둘러싸인 'the prince'가 몇 번 나타났는지 보려면 이전 예제의 findAll() 함수를 다음과 같이 고치면 된다.

In [ ]:
name_list = bs.findAll(text='the prince')
print(len(name_list)) # 7

- limit 매개변수는 물론 findAll에만 쓰인다.
- find는 findAll을 호출하면서 limit을 1로 지정한 것과 같다.
- 이 매개변수는 페이지의 항목 처음 몇 개에만 관심이 있을 때 사용한다.
- 페이지에 나타난 순서대로 찾으며 그 순서가 원하는 바와 일치한다는 보장은 없으므로 주의해야 한다.

In [ ]:
# keyword 매개변수는 특정 속성이 포함된 태그를 선택할 때 사용한다.
title = bs.findAll(id='title', class_='text')

### 2.2.2 기타 BeautifulSoup 객체
---

### Tag 객체
- 리스트 호출 또는 BeautifulSoup 객체에 find와 findAll을 호출해서 또는 다음과 같이 탐색해 들어가서 얻는다.

In [5]:
bs.div.h1

NameError: name 'bs' is not defined

- 두 가지 객체가 더 있다. 널리 쓰이지는 않지만 알아둘 가치는 있다.

### NavigableString 객체
- 태그 자체가 아니라 태그 안에 들어있는 텍스트를 나타낸다.
- 일부 함수는 NavigableStrings를 다루거나 반환한다.

### Comment 객체
- 주석 태그 안에 들어있는 HTML주석을 찾는 데 사용한다.

## 2.2.3 트리 이동
---
- findAll 함순느 이름과 속성에 따라 태그를 찾는다.
- 하지만 문서 안에서의 위치를 기준으로 태그를 찾을 때는 어떻게 해야 할까? 이럴 때 내비게이션이 필요하다.
`bs.tag.subTag.anotherSubTag`

In [ ]:
# 이 코드는 giftList 테이블에 들어 있는 제품 행 목록을(열 제목이 들어 있는 첫 행을 포함) 출력하였다.
# children() 대신 descendants() 함수를 사용했다면  테이블에 포함된 태그가 20개 이상 출력됐을 테고, 거기에는 img, span, td 태그 등이 모두 포함됐을 겁니다.
from urllib.request import urlopen
from bs4 import BeautifulStoneSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

for child in bs.find('table', {'id': 'giftList'}).children:
    print(child)

### 형제 다루기
---
BeautifulSoup의 next_siblings() 함수는 테이블에서 데이터를 쉽게 구할 수 있으며, 특히 테이블에 타이틀 행이 있을 때 유용하다.

In [ ]:
from urllib.request import urlopen
from bs4 immport BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

for silbling in bs.find('table', {'id': 'giftList'}).tr.next_siblings:
    print(sibling)

## 부모 다루기
---
- 페이지를 스크랩하다 보면, 자식이나 형제가 아니라 아주 갂믕느 부모를 찾아야할 때도 있다.
- 일반적으로 HTML 페이지에서 데이터를 수집할 목적으로 살펴볼 때는 보통 맨 위 계층에서 시작해 원하는 데이터까지 어떻게 찾아 들어갈지 생각하기 마련이다.
- 하지만 가끔 BeautifulSoup의 부모 검색 함수 .parent와 .parents가 필요할 때도 있다.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
print(bs.find('img', {'src': '../img/gifts/img1.jpg'})).parent.previous_sibling.get_text()

- 이 코드는 ../img/gifts/img1.jpg 이미지가 나타내는 객체의 가격(이 경우 $15.00)을 출력한다.
- HTML 페이지에서 우리가 살펴볼 부분의 트리 구조를 숫자로 표시한 단계와 함께 나타내면 다음과 같다.
- <tr>
- <td>
- <td>
    - "$15.00"
- s<td>
    - <img src=../img/gifts/img1.jpg''>
1. src="../img/gifts/img1.jpg"에 해당하는 이미지를 선택한다.
2. 부모 태그(이 경우 <td>태그)를 선택한다.
3. 2에서 선택한 <td>의 previous_sibling(이 경우 제품 가격이 들어있는 )

## 2.3 정규 표현식
---
- 정규 표현식이라는 이름은 정규 문자열을 식별하는 데 쓰이는 데서 유래했다. 
- 즉, 정규 표현식은 문자열이 주어진 규칙에 일치하는지, 일치하지 않는지 판단할 수 있다.
- 정규 표현식은 긴 문서에서 전화번호나 이메일 주소 같은 문자열을 빠르게 찾아보려 할 때 유용하다.